## CT and RT Structure data Pre-processing for multi-label survival prediction using planing CT

### 1. dicom to nii conversion

In [3]:
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs
import os, sys, glob
import numpy as np
#from tqdm.notebook import tqdm
from time import sleep


Please cite:
Thomas Phil, Thomas Albrecht, Skylar Gay, & Mathis Ersted Rasmussen. (2023). Sikerdebaard/dcmrtstruct2nii: dcmrtstruct2nii v5 (Version v5). Zenodo. https://doi.org/10.5281/zenodo.4037864



In [4]:
oriPath = '../../data/HNSCC/HNSCC/'
patients = os.listdir(oriPath)

savePath = '../../data/HNSCC/HNSCC_Nii_v2'
if not os.path.exists(savePath):
   os.makedirs(savePath)

In [5]:
len(patients)

627

In [14]:
for p in patients:
    if int(p.split('-')[-1]) < 130: continue
    folder = os.listdir(os.path.join(oriPath, p))
    print('++++++++++++++++++++++++++++++')
    print(p)
    print('------------------------------')
    if len(folder)==0:
        print('!!!!! No data for ',p)
        print('------------------------------')
        continue
    for f in folder:
        print(f'    folder {f}')
        print('------------------------------')
        subFolder = os.listdir(os.path.join(oriPath, p, f))
        if len(subFolder)<2:
            print(f'    !!!!! Not enough data for {p}: {f}')
            print('------------------------------')
            continue
        pSavePath = os.path.join(savePath,p)
        RTfile = []
        imagePath = None
        for s in subFolder:
            print(f'        subfolder: {s}')
            print('------------------------------')
            if len(os.listdir(os.path.join(oriPath, p, f, s)))==1:
                RTfile.append(glob.glob(os.path.join(oriPath, p, f, s,'*.dcm')))
            else:
                imagePath = os.path.join(oriPath, p, f, s)
                print(f'        image path at subfolder {s}')
                print('------------------------------')
        # dcmrtstruct2nii(RTfile[0],imagePath,pSavePath)
        RTfile = [rt for sub in RTfile for rt in sub]
        if len(RTfile) < 1:
            print(f'        !!!!! No RTStruct for folder {f}')
            print('------------------------------')
        if not imagePath: continue
        for i, rt in enumerate(RTfile):
            try:
                list_rt_structs(rt)
            except:
                print(f'        !!!!! Not a valid RTStruct: {rt}')
                continue
            print(f'        RTStruct at {rt} {i}')
            print('------------------------------')
            try:
                dcmrtstruct2nii(RTfile[i],imagePath,pSavePath+f'_0{i}')
                print(f"        Img and RTStructure Niis Generated for {p}_0{i}")
            except:
                print(f'        !!!!!!!!!!!!!!! Something wrong with {p}_0{i}')
                os.rmdir(pSavePath+f'_0{i}')

++++++++++++++++++++++++++++++
HNSCC-01-0130
------------------------------
    folder 03-25-2000-NA-MRI ORBFACEANDOR NECK WWO C-35339
------------------------------
    !!!!! Not enough data for HNSCC-01-0130: 03-25-2000-NA-MRI ORBFACEANDOR NECK WWO C-35339
------------------------------
    folder 03-25-2000-NA-PETCT HEADNECK CA IN-84763
------------------------------
    !!!!! Not enough data for HNSCC-01-0130: 03-25-2000-NA-PETCT HEADNECK CA IN-84763
------------------------------
    folder 05-22-2000-NA-MRI ORBFACEANDOR NECK WWO C-31755
------------------------------
    !!!!! Not enough data for HNSCC-01-0130: 05-22-2000-NA-MRI ORBFACEANDOR NECK WWO C-31755
------------------------------
    folder 05-29-2000-NA-RT SIMULATION-72920
------------------------------
        subfolder: 1.000000-NA-32025
------------------------------
        subfolder: 1.000000-NA-34461
------------------------------
        subfolder: 1.000000-NA-84985
------------------------------
        subfolde

In [15]:
savedPatients = os.listdir(savePath)
len(savedPatients)

594

In [5]:
savedPatients

['HNSCC-01-0216',
 'HNSCC-01-0218',
 'HNSCC-01-0219',
 'HNSCC-01-0220',
 'HNSCC-01-0221',
 'HNSCC-01-0222',
 'HNSCC-01-0223',
 'HNSCC-01-0224',
 'HNSCC-01-0225',
 'HNSCC-01-0226',
 'HNSCC-01-0227',
 'HNSCC-01-0228',
 'HNSCC-01-0229',
 'HNSCC-01-0230',
 'HNSCC-01-0231',
 'HNSCC-01-0232',
 'HNSCC-01-0233',
 'HNSCC-01-0234',
 'HNSCC-01-0235',
 'HNSCC-01-0236',
 'HNSCC-01-0237',
 'HNSCC-01-0238',
 'HNSCC-01-0239',
 'HNSCC-01-0240',
 'HNSCC-01-0241',
 'HNSCC-01-0242',
 'HNSCC-01-0243',
 'HNSCC-01-0244',
 'HNSCC-01-0245',
 'HNSCC-01-0246',
 'HNSCC-01-0247',
 'HNSCC-01-0248',
 'HNSCC-01-0249',
 'HNSCC-01-0250',
 'HNSCC-01-0251',
 'HNSCC-01-0252',
 'HNSCC-01-0253',
 'HNSCC-01-0254',
 'HNSCC-01-0255',
 'HNSCC-01-0256',
 'HNSCC-01-0257',
 'HNSCC-01-0258',
 'HNSCC-01-0259',
 'HNSCC-01-0260',
 'HNSCC-01-0261',
 'HNSCC-01-0262',
 'HNSCC-01-0263',
 'HNSCC-01-0264',
 'HNSCC-01-0265',
 'HNSCC-01-0266',
 'HNSCC-01-0267',
 'HNSCC-01-0268',
 'HNSCC-01-0269',
 'HNSCC-01-0271',
 'HNSCC-01-0272',
 'HNSCC-01

### 2. Figure out how many patient data are available

In [11]:
missing_gtv = []
missing_img = []
for p in savedPatients:
    path = os.path.join(savePath, p)
    files = os.listdir(path)

    if 'image.nii.gz' in files:
        if 'mask_GTVp.nii.gz' in files:
            print(p, files)
            continue
        else:
            missing_gtv.append(p)
            print('This patient dosen\'t have GTVp', p)
            
    else:
        missing_img.append(p)
        print('This patient dosen\'t have img', p)

This patient dosen't have img HNSCC-01-0005
This patient dosen't have img HNSCC-01-0006
This patient dosen't have img HNSCC-01-0007
This patient dosen't have img HNSCC-01-0009
This patient dosen't have GTVp HNSCC-01-0010
This patient dosen't have img HNSCC-01-0013
This patient dosen't have img HNSCC-01-0017
This patient dosen't have img HNSCC-01-0018
This patient dosen't have GTVp HNSCC-01-0028
This patient dosen't have img HNSCC-01-0029
This patient dosen't have GTVp HNSCC-01-0031
This patient dosen't have img HNSCC-01-0036
This patient dosen't have img HNSCC-01-0041
This patient dosen't have img HNSCC-01-0042
This patient dosen't have GTVp HNSCC-01-0045
This patient dosen't have GTVp HNSCC-01-0047
This patient dosen't have img HNSCC-01-0048
This patient dosen't have GTVp HNSCC-01-0049
This patient dosen't have img HNSCC-01-0051
This patient dosen't have img HNSCC-01-0056
This patient dosen't have img HNSCC-01-0060
This patient dosen't have img HNSCC-01-0065
This patient dosen't have 

In [7]:
len(missing_gtv)

20

In [8]:
len(missing_img)

37

In [10]:
avaliable_patient = list(set(savedPatients) - set(missing_gtv) - set(missing_img))
len(avaliable_patient)

398

In [10]:
from pathlib import Path
from multiprocessing import Pool
import logging

import click
import pandas as pd
import numpy as np
import SimpleITK as sitk

In [16]:
spacing = []
name = []
for p in avaliable_patient:
    try:
        ct = sitk.ReadImage(os.path.join(savePath, p, 'image.nii.gz'))
        spacing.append(ct.GetSpacing())
        name.append(p)
    except:
        print(p)

In [24]:
file = open('spacing.txt','w')
for item1,item2,item3,  in spacing:
    file.write(str(item1)+str(item2)+str(item3)+"\n")
file.close()

file = open('name.txt','w')
for item in name:
    file.write(item+"\n")
file.close()

### 3. Resampling

In [11]:
resample_path = '../data/HNSCC_OPC_Radiomics_Nii_Resample_222/'

In [25]:

if not os.path.exists(resample_path):
   os.makedirs(resample_path)

In [26]:
resampler = sitk.ResampleImageFilter()
resampler.SetOutputDirection([1, 0, 0, 0, 1, 0, 0, 0, 1])
resampling = [2,2,2]
resampler.SetOutputSpacing(resampling)

In [27]:
def get_bouding_boxes(ct, pt):
    """
    Get the bounding boxes of the CT and PT images.
    This works since all images have the same direction
    """

    ct_origin = np.array(ct.GetOrigin())
    pt_origin = np.array(pt.GetOrigin())

    ct_position_max = ct_origin + np.array(ct.GetSize()) * np.array(
        ct.GetSpacing())
    pt_position_max = pt_origin + np.array(pt.GetSize()) * np.array(
        pt.GetSpacing())
    return np.concatenate(
        [
            np.maximum(ct_origin, pt_origin),
            np.minimum(ct_position_max, pt_position_max),
        ],
        axis=0,
    )

In [28]:
def resample_one_patient(p):
    
    ct = sitk.ReadImage(os.path.join(savePath, p, 'image.nii.gz'))
    label = sitk.ReadImage(os.path.join(savePath, p, 'mask_GTVp.nii.gz'))
    bb = get_bouding_boxes(ct, ct)
    size = np.round((bb[3:] - bb[:3]) / resampling).astype(int)
    resampler.SetOutputOrigin(bb[:3])
    resampler.SetSize([int(k) for k in size])  # sitk is so stupid
    resampler.SetInterpolator(sitk.sitkBSpline)
    ct = resampler.Execute(ct)
    sitk.WriteImage(ct, os.path.join(resample_path, p+'_image.nii.gz'))
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    label = resampler.Execute(label)
    sitk.WriteImage(label, os.path.join(resample_path, p+'_mask_GTVp.nii.gz'))

In [29]:
for p in avaliable_patient:
    resample_one_patient(p)

### 4. Cropping

In [17]:
def find_centroid(mask):

    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(mask)
    try:
        centroid_coords = stats.GetCentroid(255)
    except:
        print('Something wrong')
    centroid_idx = mask.TransformPhysicalPointToIndex(centroid_coords)

    return np.asarray(centroid_idx, dtype=np.float64)

In [25]:
crop_path = '../data/HNSCC_OPC_Radiomics_Nii_Resample_222_80_80_48_Crop/'
if not os.path.exists(crop_path):
   os.makedirs(crop_path)

In [26]:
exclude_patients = ["HNSCC-01-0358","HNSCC-01-0253","HNSCC-01-0464"] # no clinic data

def tune_range(min_d, max_d, d, size_d, p):
    if min_d<0:
        min_d = 0
        max_d = min_d + d
        assert (max_d<size_d), f"Cannot extract the patch with the shape {size_d} from the image with the shape {d} for patient{p}."
    
    if max_d>d:
        max_d = d
        min_d = max_d - size_d
        assert (min_d>0), f"Cannot extract the patch with the shape {size_d} from the image with the shape {d} for patient{p}."

    return min_d, max_d

for p in avaliable_patient:

    image = sitk.ReadImage(os.path.join(resample_path, p+'_image.nii.gz'))
    mask = sitk.ReadImage(os.path.join(resample_path, p+'_mask_GTVp.nii.gz'))
    try:
        if not p in exclude_patients:
            patch_size = np.array([80,80,48])

            #crop the image to patch_size around the tumor center
            tumour_center = find_centroid(mask) # center of GTV
            size = patch_size
            min_coords = np.floor(tumour_center - size / 2).astype(np.int64)
            max_coords = np.floor(tumour_center + size / 2).astype(np.int64)
            min_x, min_y, min_z = min_coords
            max_x, max_y, max_z = max_coords

            (img_x, img_y, img_z)=image.GetSize()

            min_x, max_x = tune_range(min_x, max_x, img_x, size[0], p) 
            min_y, max_y = tune_range(min_y, max_y, img_y, size[1], p) 
            min_z, max_z = tune_range(min_z, max_z, img_z, size[2], p) 
            
            image = image[min_x:max_x, min_y:max_y, min_z:max_z]

            # window image intensities to [-500, 1000] HU range
            image = sitk.Clamp(image, sitk.sitkFloat32, -500, 500)
            mask = mask[min_x:max_x, min_y:max_y, min_z:max_z]

            sitk.WriteImage(image, os.path.join(crop_path, p+'_image.nii.gz'))
            sitk.WriteImage(mask, os.path.join(crop_path, p+'_mask_GTVp.nii.gz'))
        else:
            print('skip ', p)
            continue
    except:
        print(p)

HNSCC-01-0268
HNSCC-01-0498
HNSCC-01-0239
HNSCC-01-0628
HNSCC-01-0225
skip  HNSCC-01-0253
skip  HNSCC-01-0358
HNSCC-01-0251
HNSCC-01-0228
HNSCC-01-0576
HNSCC-01-0260
HNSCC-01-0227


In [38]:
import csv

with open('./data/avaliable_patient.csv','w') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    for p in avaliable_patient:
        wr.writerow([p])